# Carbon Emissions from Land Use Change (ELUC)
ELUC: Carbon Emissions from Land Use Change (ELUC), produced by the BLUE model [1]  
BLUE: Bookkeeping of land use emissions

See [Relevance of methodological choices for accounting of land use change carbon fluxes](https://agupubs.onlinelibrary.wiley.com/doi/10.1002/2014GB004997)

TODO: validate what ELUC represents in this file:
1. the simulated carbon fluxes occurring in that year (methods #1, #2 and #4)
2. or the fluxes committed by transitions occurring in that year (methods #3 and #5)  

See section 2.3.2 Data Postprocessing - Legacy Model Runs  

In order to make land use change recommendations (prescriptions) to minimize ELUC we need ELUC to represent 2.

ELUC is one of the elements of the [Global Carbon Budget](https://essd.copernicus.org/articles/14/4811/2022/essd-14-4811-2022.html) equation:

Reference:  
[1] “Estimates based on the BLUE model by Hansis, E., Davis, S., Pongratz, J. (2015). Relevance of methodological choices for accounting of land use change carbon fluxes, Global Biogeochemical Cycles, 2015, 10.1002/2014GB004997, as updated by Friedlingstein et al. (2022). Global Carbon Budget 2022, Earth System Science Data, [https://doi.org/10.5194/essd-14-4811-2022](https://doi.org/10.5194/essd-14-4811-2022)"


In [1]:
# You might want to install:
# %pip install xarray
# %pip install netcdf4
# %pip install matplotlib
# If you're on macOS, before installing Cartopy: brew install geos
# %pip install Cartopy
# %pip install global-land-mask

In [2]:
import numpy as np
import os
import pandas as pd
import requests
import xarray as xr

In [3]:
ELUC_DATASET_FILE = os.path.join("raw", "BLUE_LUH2-GCB2022_ELUC_gridded_net_1850-2021.nc")

In [4]:
# Download the data if needed
if not os.path.isfile(ELUC_DATASET_FILE):
    url = "https://syncandshare.lrz.de/dl/fiAuJz5VFgsEJ1E96mthCT/Data_GCB2022/BLUE_LUH2-GCB2022_ELUC_gridded_net_1850-2021.nc"
    response = requests.get(url)
    with open(ELUC_DATASET_FILE, 'wb') as f:
        f.write(response.content)

In [5]:
eluc_ds = xr.open_dataset(ELUC_DATASET_FILE)
eluc_ds

<xarray.Dataset>
Dimensions:    (lat: 720, lon: 1440, time: 172)
Coordinates:
  * lat        (lat) float32 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int32 1850 1851 1852 1853 1854 ... 2018 2019 2020 2021
Data variables:
    ELUC       (time, lat, lon) float32 ...
    cell_area  (lat, lon) float32 ...

In [6]:
eluc_ds.ELUC.dims

('time', 'lat', 'lon')

In [7]:
eluc_ds.ELUC.attrs

{'standard_name': 'Carbon emissions from land use change',
 'long_name': 'Carbon emissions from land use change',
 'units': 't C/ha/yr'}

In [8]:
#eluc_ds.ELUC.sel(time=2021).plot(x="lon");

In [9]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Center on america
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
ax.set_global()
eluc_ds.ELUC.sel(time=2021).plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines()

plt.savefig('cartopy_example.png')

In [10]:
# Carbon Emissions from land use change
# For London in 2021, closest cell to entered coordinates
eluc_ds.ELUC.sel(lat=51.507, lon=-0.127, time=2021, method="nearest")

In [11]:
eluc_ds.ELUC.sel(lat=51.507, lon=-0.127, time=2021, method="nearest").values

In [12]:
# Creates a 5.8G csv file
eluc_df = eluc_ds.to_dataframe()
eluc_df.to_csv("processed/eluc.csv")

In [13]:
# Be patient, eluc.csv is a 5.8g file
eluc_df = pd.read_csv("processed/eluc.csv")
eluc_df.head()

,lat,lon,time,ELUC,cell_area
0,89.875,-179.875,1850,0.0,168.59187
1,89.875,-179.875,1851,0.0,168.59187
2,89.875,-179.875,1852,0.0,168.59187
3,89.875,-179.875,1853,0.0,168.59187
4,89.875,-179.875,1854,0.0,168.59187


# LUH2: Land-Use Harmonization
See https://luh.umd.edu/  
The LUH2 model and datasets prepared for CMIP6 are described in [Hurtt et al. 2020.](https://gmd.copernicus.org/articles/13/5425/2020/gmd-13-5425-2020-discussion.html)


Land-use Harmonization (LUH) data for GCB 2022 is provided in 3 separate files, which can be downloaded from the following links (for the states, transitions, and management data layers respectively):

http://luh.umd.edu/LUH2/LUH2_GCB_2022/states.nc  
http://luh.umd.edu/LUH2/LUH2_GCB_2022/transitions.nc  
http://luh.umd.edu/LUH2/LUH2_GCB_2022/management.nc  

These files are based on the new HYDE3.3, as well as the 2021 FAO wood harvest data, for all years 850-2022.

The data files are for the years 850-2022, which keeps the file format consistent with the LUH2 data produced for CMIP6, hence the start year of 850. The LUH2-GCB2022 data will be different from the LUH2 v2h data used for CMIP6 for all years, due to the use of the new HYDE3.3 crop/grazing land dataset.

## LUH2 States

In [14]:
LUH2_GCB_2022_STATES_FILE = os.path.join("raw", "states.nc")

In [15]:
# Download the data if needed
if not os.path.isfile(LUH2_GCB_2022_STATES_FILE):
    url = "http://luh.umd.edu/LUH2/LUH2_GCB_2022/states.nc"
    response = requests.get(url)
    with open(LUH2_GCB_2022_STATES_FILE, 'wb') as f:
        f.write(response.content)

In [16]:
# NOTE: this is a 6.2G file
states_ds = xr.open_dataset(LUH2_GCB_2022_STATES_FILE, decode_times=False)
states_ds

<xarray.Dataset>
Dimensions:  (time: 1173, lon: 1440, lat: 720)
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 ... 1.17e+03 1.171e+03 1.172e+03
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
Data variables: (12/14)
    primf    (time, lat, lon) float32 ...
    primn    (time, lat, lon) float32 ...
    secdf    (time, lat, lon) float32 ...
    secdn    (time, lat, lon) float32 ...
    urban    (time, lat, lon) float32 ...
    c3ann    (time, lat, lon) float32 ...
    ...       ...
    c4per    (time, lat, lon) float32 ...
    c3nfx    (time, lat, lon) float32 ...
    pastr    (time, lat, lon) float32 ...
    range    (time, lat, lon) float32 ...
    secmb    (time, lat, lon) float32 ...
    secma    (time, lat, lon) float32 ...
Attributes: (12/25)
    CDI:                     Climate Data Interface version 1.9.10 (https://m...
    Conventions:             CF-1.6
    institution:             University of Maryland (UofMD), College Park, MD...
    host:                    UMD College Park
    contact:                 gchurtt@umd.edu, lchini@umd.edu, steve.frolking@...
    creation_date:           2022-08-04T11:07:54Z
    ...                      ...
    title:                   UofMD LUH2-GCB2022 dataset prepared for Global C...
    activity_id:             Global Carbon Budget
    dataset_version_number:  LUH2-GCB2022
    source_id:               UofMD-landState-LUH2-GCB2022
    target_mip:              Global Carbon Budget
    references:              Hurtt et al. 2020, Chini et al. 2021

### Dimensions
```
3 dimensions:
    time  Size:1173   *** is unlimited *** 
        axis: T
        calendar: 365_day
        long_name: time
        standard_name: time
        units: years since 850-01-01 0:0:0
    lon  Size:1440 
        standard_name: longitude
        long_name: longitude
        units: degrees_east
        axis: X
    lat  Size:720 
        standard_name: latitude
        long_name: latitude
        units: degrees_north
        axis: Y
```
### Variables
12 land types that sum to 1? Plus secmb (secondary mean biomass carbon density) and scma (secondary mean age)

1. **primf**: primary vegetation on LUH2 potential forest land
1. **primn**: primary vegetation on LUH2 potential non-forest land
1. **secdf**: secondary vegetation on LUH2 potential forest land
1. **secdn**: secondary vegetation on LUH2 potential non-forest land
1. **urban**: urban land
1. **c3ann**: C3 annual crops
1. **c4ann**: C4 annual crops
1. **c3per**: C3 perennial crops
1. **c4per**: C4 perennial crops
1. **c3nfx**: C3 nitrogen-fixing crops
1. **pastr**: managed pasture
1. **range**: rangeland
1. **secmb**: secondary mean biomass carbon density (units: kg m-2)
1. **secma**: secondary mean age


In [17]:
states_ds.sel(lat=51.507, lon=-0.127, time=2021, method="nearest")

In [18]:
# Convert the time coordinate to integer, to match eluc_ds
states_ds['time'] = states_ds['time'].astype(int)

In [19]:
# Add 850 as the time unit is "years since 850-01-01 0:0:0"
states_ds = states_ds.assign_coords(time=states_ds.time + 850)

In [20]:
states_ds

<xarray.Dataset>
Dimensions:  (time: 1173, lon: 1440, lat: 720)
Coordinates:
  * time     (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
Data variables: (12/14)
    primf    (time, lat, lon) float32 ...
    primn    (time, lat, lon) float32 ...
    secdf    (time, lat, lon) float32 ...
    secdn    (time, lat, lon) float32 ...
    urban    (time, lat, lon) float32 ...
    c3ann    (time, lat, lon) float32 ...
    ...       ...
    c4per    (time, lat, lon) float32 ...
    c3nfx    (time, lat, lon) float32 ...
    pastr    (time, lat, lon) float32 ...
    range    (time, lat, lon) float32 ...
    secmb    (time, lat, lon) float32 ...
    secma    (time, lat, lon) float32 ...
Attributes: (12/25)
    CDI:                     Climate Data Interface version 1.9.10 (https://m...
    Conventions:             CF-1.6
    institution:             University of Maryland (UofMD), College Park, MD...
    host:                    UMD College Park
    contact:                 gchurtt@umd.edu, lchini@umd.edu, steve.frolking@...
    creation_date:           2022-08-04T11:07:54Z
    ...                      ...
    title:                   UofMD LUH2-GCB2022 dataset prepared for Global C...
    activity_id:             Global Carbon Budget
    dataset_version_number:  LUH2-GCB2022
    source_id:               UofMD-landState-LUH2-GCB2022
    target_mip:              Global Carbon Budget
    references:              Hurtt et al. 2020, Chini et al. 2021

In [21]:
states_ds.sel(lat=51.507, lon=-0.127, time=2021, method="nearest")

## LUH2 Transitions

Exploring the transitions file. Not used for the moment

In [22]:
LUH2_GCB_2022_TRANSITIONS_FILE = os.path.join("raw", "transitions.nc")

In [23]:
# Download the data if needed
# NOTE: this a 19G file
if not os.path.isfile(LUH2_GCB_2022_TRANSITIONS_FILE):
    url = "http://luh.umd.edu/LUH2/LUH2_GCB_2022/transitions.nc"
    response = requests.get(url)
    with open(LUH2_GCB_2022_TRANSITIONS_FILE, 'wb') as f:
        f.write(response.content)

In [24]:
transitions_ds = xr.open_dataset(LUH2_GCB_2022_TRANSITIONS_FILE, decode_times=False)
transitions_ds

### Variables
12 land types  
No transition to `primf` and `primn`  
12 * 9 = 108 transitions  
5 transitions to `harv`  
5 transitions to `bioh`  
= 118 transitions

# Aggregated dataset
Merge the ELUC and states datasets

In [25]:
# Merge
agg_ds = xr.merge([eluc_ds, states_ds])

In [26]:
agg_ds

<xarray.Dataset>
Dimensions:    (lat: 720, lon: 1440, time: 1173)
Coordinates:
  * lat        (lat) float32 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
Data variables: (12/16)
    ELUC       (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    cell_area  (lat, lon) float32 ...
    primf      (time, lat, lon) float32 ...
    primn      (time, lat, lon) float32 ...
    secdf      (time, lat, lon) float32 ...
    secdn      (time, lat, lon) float32 ...
    ...         ...
    c4per      (time, lat, lon) float32 ...
    c3nfx      (time, lat, lon) float32 ...
    pastr      (time, lat, lon) float32 ...
    range      (time, lat, lon) float32 ...
    secmb      (time, lat, lon) float32 ...
    secma      (time, lat, lon) float32 ...

In [27]:
# Drop the variables we don't need
agg_ds = agg_ds.drop_vars(["secmb", "secma"])
agg_ds

<xarray.Dataset>
Dimensions:    (lat: 720, lon: 1440, time: 1173)
Coordinates:
  * lat        (lat) float32 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
Data variables: (12/14)
    ELUC       (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    cell_area  (lat, lon) float32 ...
    primf      (time, lat, lon) float32 ...
    primn      (time, lat, lon) float32 ...
    secdf      (time, lat, lon) float32 ...
    secdn      (time, lat, lon) float32 ...
    ...         ...
    c4ann      (time, lat, lon) float32 ...
    c3per      (time, lat, lon) float32 ...
    c4per      (time, lat, lon) float32 ...
    c3nfx      (time, lat, lon) float32 ...
    pastr      (time, lat, lon) float32 ...
    range      (time, lat, lon) float32 ...

In [28]:
agg_ds.sel(lat=51.507, lon=-0.127, time=2021, method="nearest").urban.values

### London, industrial period

In [29]:
london_ds = agg_ds.sel(lat=51.507, lon=-0.127, method="nearest")

In [30]:
london_indus_ds = london_ds.sel(time=(slice(1850, 2022)))

In [31]:
london_indus_ds

In [32]:
london_df = london_indus_ds.to_dataframe()
london_df

### Export 2D data set

In [33]:
agg_indus_ds = agg_ds.sel(time=(slice(1850, 2022)))

In [34]:
agg_indus_df = agg_indus_ds.to_dataframe()

In [35]:
agg_indus_df.head()

In [36]:
agg_indus_df.describe()

In [37]:
# Convert the multi-index to lat long columns
agg_indus_df = agg_indus_df.reset_index()

In [38]:
# London
agg_indus_df[(agg_indus_df["lat"]==51.625) &
             (agg_indus_df["lon"]==-0.125)]


In [39]:
# WARNING: Results in a huge file. ~12G
agg_indus_df.to_csv("processed/agg_indus.csv")

# TODO

1. Find a place to put and share the data (ITU S3?) [#14](https://github.com/Project-Resilience/mvp/issues/14)
1. Check ELUC number corresponds to emissions of the change during that 1 year [#15](https://github.com/Project-Resilience/mvp/issues/15)
1. Filter out the data: remove oceans. Lots of zeroes. [#16](https://github.com/Project-Resilience/mvp/issues/16)
1. Compute the year over year diff e.g. ELUC for 2021 corresponds to LUC between 2020 and 2021 [#17](https://github.com/Project-Resilience/mvp/issues/17)
1. Train predictor models to predict ELUC from LUC [#18](https://github.com/Project-Resilience/mvp/issues/18)
     1. Train / Test split by years: test on last 15 years
     1. Train / Test split by geographical regions
     1. Train / Test random
     1. Does it work?
1. Validate with the authors
1. Train a prescriptor
1. Build a UI to use the models (even in jupyter notebooks)

In [40]:
agg_ds

<xarray.Dataset>
Dimensions:    (lat: 720, lon: 1440, time: 1173)
Coordinates:
  * lat        (lat) float32 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
Data variables: (12/14)
    ELUC       (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    cell_area  (lat, lon) float32 ...
    primf      (time, lat, lon) float32 ...
    primn      (time, lat, lon) float32 ...
    secdf      (time, lat, lon) float32 ...
    secdn      (time, lat, lon) float32 ...
    ...         ...
    c4ann      (time, lat, lon) float32 ...
    c3per      (time, lat, lon) float32 ...
    c4per      (time, lat, lon) float32 ...
    c3nfx      (time, lat, lon) float32 ...
    pastr      (time, lat, lon) float32 ...
    range      (time, lat, lon) float32 ...

In [41]:
from global_land_mask import globe
# Make a grid
lat_grid, lon_grid = np.meshgrid(agg_ds.lat.values,agg_ds.lon.values)

# Get whether the points are on land using globe.is_land
globe_land_mask = globe.is_land(lat_grid, lon_grid)
print(np.sum(globe_land_mask))
# Add mask as coordinate to xarray
agg_ds.coords['mask'] = (('lon', 'lat'), globe_land_mask)
# Select only ones where mask is True, i.e. on land

343956


In [42]:
print(globe_land_mask.shape)
print(globe_land_mask)
# Chunk for better performance on lower resource machines
agg_ds = agg_ds.chunk({"lon": 360, "lat": 180, "time": 100})
agg_ds

(1440, 720)
[[False False False ...  True  True  True]
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]
 ...
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]
 [False False False ...  True  True  True]]


<xarray.Dataset>
Dimensions:    (lat: 720, lon: 1440, time: 1173)
Coordinates:
  * lat        (lat) float32 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
    mask       (lon, lat) bool dask.array<chunksize=(360, 180), meta=np.ndarray>
Data variables: (12/14)
    ELUC       (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    cell_area  (lat, lon) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    primf      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    primn      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    secdf      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    secdn      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    ...         ...
    c4ann      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    c3per      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    c4per      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    c3nfx      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    pastr      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>
    range      (time, lat, lon) float32 dask.array<chunksize=(100, 180, 360), meta=np.ndarray>

In [43]:
agg_ds = agg_ds.where(agg_ds.mask == 1, drop=True)

agg_ds

<xarray.Dataset>
Dimensions:    (time: 1173, lat: 671, lon: 1440)
Coordinates:
  * lat        (lat) float32 83.38 83.12 82.88 82.62 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 850 851 852 853 854 855 ... 2018 2019 2020 2021 2022
    mask       (lon, lat) bool dask.array<chunksize=(360, 154), meta=np.ndarray>
Data variables: (12/14)
    ELUC       (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    cell_area  (lat, lon) float32 dask.array<chunksize=(154, 360), meta=np.ndarray>
    primf      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    primn      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    secdf      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    secdn      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    ...         ...
    c4ann      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    c3per      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    c4per      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    c3nfx      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    pastr      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>
    range      (time, lat, lon) float32 dask.array<chunksize=(100, 154, 360), meta=np.ndarray>

In [44]:
agg_ds_diff = agg_ds.diff("time")
agg_ds_diff

<xarray.Dataset>
Dimensions:    (time: 1172, lat: 671, lon: 1440)
Coordinates:
  * lat        (lat) float32 83.38 83.12 82.88 82.62 ... -89.38 -89.62 -89.88
  * lon        (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time       (time) int64 851 852 853 854 855 856 ... 2018 2019 2020 2021 2022
    mask       (lon, lat) bool dask.array<chunksize=(360, 154), meta=np.ndarray>
Data variables: (12/14)
    ELUC       (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    cell_area  (lat, lon) float32 dask.array<chunksize=(154, 360), meta=np.ndarray>
    primf      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    primn      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    secdf      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    secdn      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    ...         ...
    c4ann      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    c3per      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    c4per      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    c3nfx      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    pastr      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>
    range      (time, lat, lon) float32 dask.array<chunksize=(99, 154, 360), meta=np.ndarray>